In [1]:
# =============================================================================
# 앙상블 모델 (RF + XGBoost + FT Transformer)
# =============================================================================

# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

# 라이브러리 설치 및 임포트
!pip install optuna -q

import pickle
import pandas as pd
import numpy as np
import optuna
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


# 1. 데이터 전처리

## 1.1. 데이터 로드

In [3]:
print("="*60)
print("📂 데이터 로드")
print("="*60)

DATA_PATH = '/content/drive/MyDrive/preprocessed_data.pkl'

with open(DATA_PATH, 'rb') as f:
    data_dict = pickle.load(f)

X_test = data_dict['X_test_clf']
y_test = data_dict['y_test_clf']

print(f"✅ Test: {X_test.shape}")

📂 데이터 로드
✅ Test: (9670, 45)


## 1.2. RandomForest 모델을 위한 데이터 전처리

In [4]:
def create_features(df):
    data = df.copy()

    if 'outstanding_debt' in data.columns and 'annual_income' in data.columns:
        data['debt_to_income'] = data['outstanding_debt'] / (data['annual_income'] + 1)
    if 'total_emi_per_month' in data.columns and 'monthly_inhand_salary' in data.columns:
        data['emi_to_income'] = data['total_emi_per_month'] / (data['monthly_inhand_salary'] + 1)
    if 'monthly_balance' in data.columns and 'monthly_inhand_salary' in data.columns:
        data['balance_to_income'] = data['monthly_balance'] / (data['monthly_inhand_salary'] + 1)
    if 'amount_invested_monthly' in data.columns and 'monthly_inhand_salary' in data.columns:
        data['invest_to_income'] = data['amount_invested_monthly'] / (data['monthly_inhand_salary'] + 1)
    if 'num_credit_card' in data.columns and 'credit_utilization_ratio' in data.columns:
        data['credit_load'] = data['num_credit_card'] * data['credit_utilization_ratio']
    if 'delay_from_due_date' in data.columns and 'num_of_delayed_payment' in data.columns:
        data['delay_severity'] = data['delay_from_due_date'] * data['num_of_delayed_payment']
    if 'num_of_loan' in data.columns and 'num_bank_accounts' in data.columns:
        data['loan_per_account'] = data['num_of_loan'] / (data['num_bank_accounts'] + 1)
    if 'num_credit_inquiries' in data.columns and 'credit_history_age' in data.columns:
        data['inquiries_per_year'] = data['num_credit_inquiries'] / (data['credit_history_age'] / 12 + 1)
    if 'num_bank_accounts' in data.columns and 'credit_history_age' in data.columns:
        data['accounts_per_year'] = data['num_bank_accounts'] / (data['credit_history_age'] / 12 + 1)
    if 'total_emi_per_month' in data.columns and 'outstanding_debt' in data.columns and 'interest_rate' in data.columns:
        data['payment_burden'] = data['total_emi_per_month'] + data['outstanding_debt'] * (data['interest_rate'] / 100)
    if 'annual_income' in data.columns:
        data['log_income'] = np.log1p(data['annual_income'])
    if 'outstanding_debt' in data.columns:
        data['log_debt'] = np.log1p(data['outstanding_debt'])

    return data

# 2. RandomForest

In [7]:
# 1. Random Forest 로드
RF_PATH = '/content/drive/MyDrive/best_rf_model.pkl'
with open(RF_PATH, 'rb') as f:
    rf_dict = pickle.load(f)
rf_model = rf_dict['model']
rf_top_features = rf_dict['top_features']
print(f"✅ RF 로드 완료 (피처: {len(rf_top_features)}개)")

✅ RF 로드 완료 (피처: 20개)


# 3. XGBoost

In [8]:
# 2. XGBoost 로드
XGB_PATH = '/content/drive/MyDrive/best_xgb_model.pkl'
with open(XGB_PATH, 'rb') as f:
    xgb_dict = pickle.load(f)
xgb_model = xgb_dict['model']
print(f"✅ XGBoost 로드 완료 (원본 피처)")

✅ XGBoost 로드 완료 (원본 피처)


# 4. FT Transformer

In [9]:
# 3. FT Transformer 로드
FT_PATH = '/content/drive/MyDrive/best_ft_model.pkl'
with open(FT_PATH, 'rb') as f:
    ft_dict = pickle.load(f)

class FTTransformer(nn.Module):
    def __init__(self, input_dim, num_classes, embed_dim=64, num_heads=4,
                 num_layers=3, ff_dim=128, dropout=0.1):
        super().__init__()
        self.feature_embedding = nn.Linear(1, embed_dim)
        self.input_dim = input_dim
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.pos_embedding = nn.Parameter(torch.randn(1, input_dim + 1, embed_dim))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(embed_dim)
        self.fc = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(ff_dim, num_classes)
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = x.unsqueeze(-1)
        x = self.feature_embedding(x)
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat([cls_tokens, x], dim=1)
        x = x + self.pos_embedding
        x = self.transformer(x)
        cls_output = x[:, 0]
        cls_output = self.norm(cls_output)
        return self.fc(cls_output)

ft_params = ft_dict['best_params']
ft_model = FTTransformer(
    input_dim=ft_dict['input_dim'],
    num_classes=3,
    embed_dim=ft_params['embed_dim'],
    num_heads=ft_params['num_heads'],
    num_layers=ft_params['num_layers'],
    ff_dim=ft_params['ff_dim'],
    dropout=ft_params['dropout']
).to(device)
ft_model.load_state_dict(ft_dict['model_state'])
ft_model.eval()
ft_scaler = ft_dict['scaler']
print(f"✅ FT Transformer 로드 완료 (원본 피처)")

✅ FT Transformer 로드 완료 (원본 피처)


# 5. Ensemble Model (Soft Voting)

In [10]:
# =============================================================================
# 각 모델별 데이터 준비 및 예측
# =============================================================================
print("\n" + "="*60)
print("🔮 예측")
print("="*60)

# 1. RF 예측 (피처 엔지니어링 + Feature Selection)
X_test_fe = create_features(X_test)
X_test_rf = X_test_fe[rf_top_features]
rf_pred = rf_model.predict(X_test_rf)
rf_proba = rf_model.predict_proba(X_test_rf)
print(f"✅ RF 예측 완료")

# 2. XGBoost 예측 (원본 데이터)
xgb_pred = xgb_model.predict(X_test)
xgb_proba = xgb_model.predict_proba(X_test)
print(f"✅ XGBoost 예측 완료")

# 3. FT Transformer 예측 (원본 데이터 + 스케일링)
X_test_scaled = ft_scaler.transform(X_test)
X_test_t = torch.FloatTensor(X_test_scaled).to(device)

with torch.no_grad():
    ft_logits = ft_model(X_test_t)
    ft_pred = torch.argmax(ft_logits, dim=1).cpu().numpy()
    ft_proba = torch.softmax(ft_logits, dim=1).cpu().numpy()
print(f"✅ FT Transformer 예측 완료")


🔮 예측
✅ RF 예측 완료
✅ XGBoost 예측 완료
✅ FT Transformer 예측 완료


## 5.1. 앙상블 모델

In [11]:
# =============================================================================
# 앙상블 (Soft Voting)
# =============================================================================
print("\n" + "="*60)
print("🏆 앙상블 (Soft Voting)")
print("="*60)

# 가중치 설정 (조정 가능)
weights = [0.3, 0.35, 0.35]  # RF, XGB, FT
print(f"가중치: RF={weights[0]}, XGB={weights[1]}, FT={weights[2]}")

ensemble_proba = weights[0] * rf_proba + weights[1] * xgb_proba + weights[2] * ft_proba
ensemble_pred = np.argmax(ensemble_proba, axis=1)

# =============================================================================
# 결과 출력
# =============================================================================
print("\n" + "="*60)
print("📊 개별 모델 성능")
print("="*60)

print(f"Random Forest  - Acc: {accuracy_score(y_test, rf_pred):.4f}, F1: {f1_score(y_test, rf_pred, average='macro'):.4f}")
print(f"XGBoost        - Acc: {accuracy_score(y_test, xgb_pred):.4f}, F1: {f1_score(y_test, xgb_pred, average='macro'):.4f}")
print(f"FT Transformer - Acc: {accuracy_score(y_test, ft_pred):.4f}, F1: {f1_score(y_test, ft_pred, average='macro'):.4f}")

print("\n" + "="*60)
print("🎯 앙상블 성능")
print("="*60)

print(f"Accuracy: {accuracy_score(y_test, ensemble_pred):.4f}")
print(f"F1 Score (macro): {f1_score(y_test, ensemble_pred, average='macro'):.4f}")

print("\n📋 Classification Report (앙상블):")
print(classification_report(y_test, ensemble_pred, target_names=['Bad', 'Standard', 'Good']))


🏆 앙상블 (Soft Voting)
가중치: RF=0.3, XGB=0.35, FT=0.35

📊 개별 모델 성능
Random Forest  - Acc: 0.7967, F1: 0.7850
XGBoost        - Acc: 0.8296, F1: 0.8239
FT Transformer - Acc: 0.7347, F1: 0.7179

🎯 앙상블 성능
Accuracy: 0.8084
F1 Score (macro): 0.7997

📋 Classification Report (앙상블):
              precision    recall  f1-score   support

         Bad       0.80      0.79      0.79      2749
    Standard       0.83      0.82      0.83      5150
        Good       0.75      0.81      0.78      1771

    accuracy                           0.81      9670
   macro avg       0.80      0.80      0.80      9670
weighted avg       0.81      0.81      0.81      9670



In [12]:
# =============================================================================
# 비교 테이블
# =============================================================================
print("\n" + "="*60)
print("📊 최종 결과 비교")
print("="*60)

results = pd.DataFrame({
    'Model': ['Random Forest', 'XGBoost', 'FT Transformer', '🏆 Ensemble'],
    'Accuracy': [
        accuracy_score(y_test, rf_pred),
        accuracy_score(y_test, xgb_pred),
        accuracy_score(y_test, ft_pred),
        accuracy_score(y_test, ensemble_pred)
    ],
    'F1 (macro)': [
        f1_score(y_test, rf_pred, average='macro'),
        f1_score(y_test, xgb_pred, average='macro'),
        f1_score(y_test, ft_pred, average='macro'),
        f1_score(y_test, ensemble_pred, average='macro')
    ]
})

print(results.to_string(index=False))


📊 최종 결과 비교
         Model  Accuracy  F1 (macro)
 Random Forest  0.796691    0.785042
       XGBoost  0.829576    0.823887
FT Transformer  0.734747    0.717948
    🏆 Ensemble  0.808376    0.799707


## 5.2. 앙상블 모델 가중치 튜닝

In [13]:
print("\n" + "="*60)
print("🔧 최적 가중치 탐색")
print("="*60)

best_f1 = 0
best_weights = None

for w1 in np.arange(0.1, 0.6, 0.1):
    for w2 in np.arange(0.1, 0.6, 0.1):
        w3 = 1 - w1 - w2
        if w3 < 0.1:
            continue

        ensemble_proba_temp = w1 * rf_proba + w2 * xgb_proba + w3 * ft_proba
        ensemble_pred_temp = np.argmax(ensemble_proba_temp, axis=1)
        f1_temp = f1_score(y_test, ensemble_pred_temp, average='macro')

        if f1_temp > best_f1:
            best_f1 = f1_temp
            best_weights = (w1, w2, w3)

print(f"✅ 최적 가중치: RF={best_weights[0]:.1f}, XGB={best_weights[1]:.1f}, FT={best_weights[2]:.1f}")
print(f"✅ 최적 F1 Score: {best_f1:.4f}")

# 최적 가중치로 최종 예측
final_proba = best_weights[0] * rf_proba + best_weights[1] * xgb_proba + best_weights[2] * ft_proba
final_pred = np.argmax(final_proba, axis=1)

print(f"\n🎯 최적 가중치 앙상블 성능:")
print(f"   Accuracy: {accuracy_score(y_test, final_pred):.4f}")
print(f"   F1 Score (macro): {f1_score(y_test, final_pred, average='macro'):.4f}")


🔧 최적 가중치 탐색
✅ 최적 가중치: RF=0.3, XGB=0.5, FT=0.2
✅ 최적 F1 Score: 0.8131

🎯 최적 가중치 앙상블 성능:
   Accuracy: 0.8199
   F1 Score (macro): 0.8131
